<font size=6> The total sovereign emissions calculated by adding up scope 1, 2 and 3 emissions  </font>

In [1]:
report_currency = 'USD'

In [2]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

#trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [4]:
import boto3
import pandas as pd

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [5]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)

ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_total_sovereign_emissions'

src_table_1 = 'sf_oecd_imgr_fco2'
src_table_2 = 'sf_unfccc_results'
src_table_3 = 'sf_oecd_exgr_dco2'
src_table_4 = 'sf_oecd_exch_rates'
src_table_5 = 'sf_wdi_population'




In [6]:
import pandas as pd
from openscm_units import unit_registry
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
# First we create the registry.
ureg = unit_registry
Q_ = ureg.Quantity
ureg.default_format = '~'
ureg.define("CO2e = CO2 = CO2eq")
ureg.define("USD = [currency] ")
#ureg.define("EUR = [currency_EUR] ")
ureg.define('Millions=1000000')
set_application_registry(ureg)


In [7]:
def requantify_df(df):
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df


In [8]:
# If DF_COL contains Pint quantities (because it is a PintArray or an array of Pint Quantities),
# return a two-column dataframe of magnitudes and units.
# If DF_COL contains no Pint quanities, return it unchanged.

def dequantify_column(df_col: pd.Series):
    if type(df_col.values)==PintArray:
        return pd.DataFrame({df_col.name: df_col.values.quantity.m,
                             df_col.name + "_units": str(df_col.values.dtype.units)},
                            index=df_col.index)
    elif df_col.size==0:
        return df_col
    elif isinstance(df_col.iloc[0], Quantity):
        values = df_col.map(lambda x: (x.m, x.u))
        return pd.DataFrame({df_col.name: df_col.map(lambda x: x.m),
                             df_col.name + "_units": df_col.map(lambda x: str(x.u))},
                            index=df_col.index)
    else:
        return df_col

# Rewrite dataframe DF so that columns containing Pint quantities are represented by a column for the Magnitude and column for the Units.
# The magnitude column retains the original column name and the units column is renamed with a _units suffix.
def dequantify_df(df):
    return pd.concat([dequantify_column(df[col]) for col in df.columns], axis=1)

In [9]:
# Get the values saved in Trino related with Foreign CO2 emissions embodied in gross imports

import pandas as pd

sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1} """ 
df1 = pd.read_sql(sql, engine)
df1 = requantify_df(df1)
df1
# convert to CO2 units to t
df1['value'] = df1['value'].pint.to("t CO2e")
df1 = dequantify_df(df1)
df1

,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,NOR,PHL,DTOTAL,2011,Foreign CO2 emissions embodied in gross imports,90000.0,CO2eq * t
1,NOR,PHL,DTOTAL,2012,Foreign CO2 emissions embodied in gross imports,70000.0,CO2eq * t
2,NOR,PHL,DTOTAL,2013,Foreign CO2 emissions embodied in gross imports,80000.0,CO2eq * t
3,NOR,PHL,DTOTAL,2014,Foreign CO2 emissions embodied in gross imports,76000.0,CO2eq * t
4,NOR,PHL,DTOTAL,2015,Foreign CO2 emissions embodied in gross imports,72000.0,CO2eq * t
...,...,...,...,...,...,...,...
334651,CHN,TWN,D35,1998,Foreign CO2 emissions embodied in gross imports,8000.0,CO2eq * t
334652,CHN,TWN,D35,1999,Foreign CO2 emissions embodied in gross imports,9000.0,CO2eq * t
334653,CHN,TWN,D35,2000,Foreign CO2 emissions embodied in gross imports,11000.0,CO2eq * t
334654,CHN,TWN,D35,2001,Foreign CO2 emissions embodied in gross imports,13000.0,CO2eq * t


In [10]:
sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_3} where industry_code='DTOTAL'""" 
df_exgr = pd.read_sql(sql, engine)
df_exgr = requantify_df(df_exgr)
# convert to CO2 units to t
df_exgr['value'] = df_exgr['value'].pint.to("t CO2e")
df_exgr = dequantify_df(df_exgr)
df_exgr

,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,OECD,WLD,DTOTAL,1995,Domestic CO2 emissions embodied in gross exports,530637000.0,CO2eq * t
1,OECD,WLD,DTOTAL,1996,Domestic CO2 emissions embodied in gross exports,580962000.0,CO2eq * t
2,OECD,WLD,DTOTAL,1997,Domestic CO2 emissions embodied in gross exports,638161000.0,CO2eq * t
3,OECD,WLD,DTOTAL,1998,Domestic CO2 emissions embodied in gross exports,605956000.0,CO2eq * t
4,OECD,WLD,DTOTAL,1999,Domestic CO2 emissions embodied in gross exports,576039000.0,CO2eq * t
...,...,...,...,...,...,...,...
167323,CRI,TUN,DTOTAL,2014,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t
167324,CRI,TUN,DTOTAL,2015,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t
167325,CRI,TUN,DTOTAL,2016,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t
167326,CRI,TUN,DTOTAL,2017,Domestic CO2 emissions embodied in gross exports,0.0,CO2eq * t


In [11]:
###########################

sql=f"""
select validity_date,value from {ingest_catalog}.{ingest_schema}.{src_table_4} where country_iso_code='DEU'""" 
df_exch_eur = pd.read_sql(sql, engine)
df_exch_eur

,validity_date,value
0,1950,2.144861
1,1951,2.144861
2,1952,2.144861
3,1953,2.147426
4,1954,2.147426
...,...,...
67,2017,0.885206
68,2018,0.846773
69,2019,0.893276
70,2020,0.875506


In [12]:
import pandas as pd 
sql=f"""
select country_iso_code,validity_date,value from {ingest_catalog}.{ingest_schema}.{src_table_5} """ 
df_population = pd.read_sql(sql, engine)
df_population["value"] = pd.to_numeric(df_population["value"], errors='coerce')
df_population["validity_date"] = pd.to_numeric(df_population["validity_date"], errors='coerce')
df_population=df_population.convert_dtypes()
df_population
df_population.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  3192 non-null   string
 1   validity_date     3192 non-null   Int64 
 2   value             3170 non-null   Int64 
dtypes: Int64(2), string(1)
memory usage: 81.2 KB


<font size=3> Remove invalid country codes </font>

In [13]:
import pycountry
import pandas as pd
df_country = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df_country = df_country.rename(columns={'alpha_3':'country_iso_code'})
df_country=df_country[['country_iso_code']]
# add Rest of World to the country dataframe
dict_row = {'country_iso_code':'ROW'}
df_country = df_country.append(dict_row, ignore_index = True)
df_country.info(verbose=True)
df1=df1.convert_dtypes()
#df1.info(verbose=True)
df1 = pd.merge(df1,df_country,on=['country_iso_code'])  
df1=df1.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df1

df_exgr = pd.merge(df_exgr,df_country,on=['country_iso_code'])  
df_exgr=df_exgr.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df_exgr






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  250 non-null    object
dtypes: object(1)
memory usage: 2.1+ KB


/tmp/ipykernel_1176/4216143356.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_country = df_country.append(dict_row, ignore_index = True)


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,TWN,ZSCA,DTOTAL,2003,Domestic CO2 emissions embodied in gross exports,1170000.0,CO2eq * t
1,TWN,ZSCA,DTOTAL,2004,Domestic CO2 emissions embodied in gross exports,1194000.0,CO2eq * t
2,TWN,ZSCA,DTOTAL,2005,Domestic CO2 emissions embodied in gross exports,1473000.0,CO2eq * t
3,TWN,ZSCA,DTOTAL,2006,Domestic CO2 emissions embodied in gross exports,1925000.0,CO2eq * t
4,TWN,ZSCA,DTOTAL,2007,Domestic CO2 emissions embodied in gross exports,2482000.0,CO2eq * t
...,...,...,...,...,...,...,...
135067,HUN,THA,DTOTAL,1997,Domestic CO2 emissions embodied in gross exports,36000.0,CO2eq * t
135068,HUN,THA,DTOTAL,1998,Domestic CO2 emissions embodied in gross exports,24000.0,CO2eq * t
135069,HUN,THA,DTOTAL,1999,Domestic CO2 emissions embodied in gross exports,23000.0,CO2eq * t
135070,HUN,THA,DTOTAL,2000,Domestic CO2 emissions embodied in gross exports,23000.0,CO2eq * t


Calcuate Scope 2:
GHG emissions occurring as a consequence of the domestic use of grid-supplied electricity, heat,
steam and/or cooling which is imported from another territory


In [14]:
rename_columns = {'value':'total','partner_iso_code':'country_iso_code','value_units' :'total_units'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df1 = df1.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df1.convert_dtypes()

df1.info(verbose=True)
df1

######################

df_exgr = df_exgr.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df_exgr.convert_dtypes()

df_exgr.info(verbose=True)
df_exgr



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         4032 non-null   string 
 1   industry_code     4032 non-null   string 
 2   country_iso_code  4032 non-null   string 
 3   validity_date     4032 non-null   Int64  
 4   total             4032 non-null   Float64
 5   total_units       4032 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 197.0 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2016 entries, 0 to 2015
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         2016 non-null   string 
 1   industry_code     2016 non-null   string 
 2   country_iso_code  2016 non-null   string 
 3   validity_date     2016 non-null   Int64  
 4   total             2016 non-null   Float64
 5   total_units    

,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1995,2008843000.0,CO2eq * t
1,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1996,2055450000.0,CO2eq * t
2,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1997,2151870000.0,CO2eq * t
3,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1998,2177243000.0,CO2eq * t
4,Domestic CO2 emissions embodied in gross exports,DTOTAL,APEC,1999,2300588000.0,CO2eq * t
...,...,...,...,...,...,...
2011,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2014,266081000.0,CO2eq * t
2012,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2015,231751000.0,CO2eq * t
2013,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2016,211686000.0,CO2eq * t
2014,Domestic CO2 emissions embodied in gross exports,DTOTAL,ZSCA,2017,226732000.0,CO2eq * t


In [15]:
# divide the dataframe into two dataframes 
# to calculate Scope2 and Scope3 Values
df_D35 = df1[df1['industry_code']=='D35']
df_D35 = df_D35[['country_iso_code','validity_date','total','total_units']]
df_DTOTAL = df1[df1['industry_code']=='DTOTAL']
df_DTOTAL = df_DTOTAL[['country_iso_code','validity_date','total','total_units']]
df_result = pd.merge(df_D35,df_DTOTAL,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()

#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total_x':'scope2_value','total_y':'scope3_value','total_units_x':'scope2_value_units','total_units_y':'scope3_value_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result['scope3_value'] = df_result['scope3_value'] - df_result['scope2_value'] 
df_result[df_result['country_iso_code']=='CAN']

df_result = pd.merge(df_result,df_exgr,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total':'exported_emissions','total_units':'exported_emissions_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result.drop(columns=['attribute','industry_code'],inplace=True)
df_result.info(verbose=True)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 2016 entries, 0 to 2015
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   country_iso_code          2016 non-null   string 
 1   validity_date             2016 non-null   Int64  
 2   scope2_value              2016 non-null   Float64
 3   scope2_value_units        2016 non-null   string 
 4   scope3_value              2016 non-null   Float64
 5   scope3_value_units        2016 non-null   string 
 6   exported_emissions        2016 non-null   Float64
 7   exported_emissions_units  2016 non-null   string 
dtypes: Float64(3), Int64(1), string(4)
memory usage: 149.6 KB


In [16]:
# Get the values saved in Trino containing  GHG emissions provided by UNFCCC & GDP PPP values from Worldbank

import pandas as pd


sql=f"""
select country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_without_lulucf_units,ghg_total_with_lulucf,ghg_total_with_lulucf_units
,gdp,gdp_units,gdp_ppp,gdp_ppp_units
from {ingest_catalog}.{ingest_schema}.{src_table_2} """ 
df_unfccc = pd.read_sql(sql, engine)

if (report_currency == 'EUR'):
    df_unfccc = pd.merge(df_unfccc,df_exch_eur,on=['validity_date'],how='inner')   
    df_unfccc['gdp'] = df_unfccc['gdp'] * df_unfccc['value']
    df_unfccc['gdp_ppp'] = df_unfccc['gdp_ppp'] * df_unfccc['value']
    df_unfccc['gdp_units'] = 'EUR'
    df_unfccc['gdp_ppp_units'] = 'EUR'
    df_unfccc.drop(columns=['value'],inplace=True)
    


df_unfccc[df_unfccc['country_iso_code']=='ARG']
df_unfccc = df_unfccc.convert_dtypes()
df_unfccc.info(verbose=True)
#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12948 entries, 0 to 12947
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                12948 non-null  string 
 1   validity_date                   12948 non-null  Int64  
 2   ghg_total_without_lulucf        2212 non-null   Float64
 3   ghg_total_without_lulucf_units  12948 non-null  string 
 4   ghg_total_with_lulucf           2212 non-null   Float64
 5   ghg_total_with_lulucf_units     12948 non-null  string 
 6   gdp                             12854 non-null  Float64
 7   gdp_units                       12948 non-null  string 
 8   gdp_ppp                         7210 non-null   Float64
 9   gdp_ppp_units                   12948 non-null  string 
dtypes: Float64(4), Int64(1), string(5)
memory usage: 1.0 MB


In [17]:
import numpy as np
df_unfccc = df_unfccc.convert_dtypes()
df_unfccc.info(verbose=True)

df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf.isnull(), 0, df_unfccc.ghg_total_without_lulucf))
df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf.isnull(), 0, df_unfccc.ghg_total_with_lulucf))
df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp.isnull(), 0, df_unfccc.gdp))
df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp.isnull(), 0, df_unfccc.gdp_ppp))

df_unfccc = requantify_df(df_unfccc)
df_unfccc.info(verbose=True)
# convert to CO2 units to Mt
df_unfccc['ghg_total_without_lulucf'] = df_unfccc['ghg_total_without_lulucf'].pint.to("t CO2e")
df_unfccc['ghg_total_with_lulucf'] = df_unfccc['ghg_total_with_lulucf'].pint.to("t CO2e")
####

#df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf==0, np.nan, df_unfccc.ghg_total_without_lulucf))
#df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf==0, np.nan, df_unfccc.ghg_total_with_lulucf))
#df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp==0, np.nan, df_unfccc.gdp))
#df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp==0, np.nan, df_unfccc.gdp_ppp))

df_unfccc.info(verbose=True)


df_unfccc[df_unfccc['country_iso_code']=='ARG']



#df_unfccc['gdp'] = df_unfccc['gdp'].pint.to("Millions USD")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12948 entries, 0 to 12947
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country_iso_code                12948 non-null  string 
 1   validity_date                   12948 non-null  Int64  
 2   ghg_total_without_lulucf        2212 non-null   Float64
 3   ghg_total_without_lulucf_units  12948 non-null  string 
 4   ghg_total_with_lulucf           2212 non-null   Float64
 5   ghg_total_with_lulucf_units     12948 non-null  string 
 6   gdp                             12854 non-null  Float64
 7   gdp_units                       12948 non-null  string 
 8   gdp_ppp                         7210 non-null   Float64
 9   gdp_ppp_units                   12948 non-null  string 
dtypes: Float64(4), Int64(1), string(5)
memory usage: 1.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12948 entries, 0 to 12947
Data columns (total 6 

/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/opt/app-root/lib64/python3.8/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,country_iso_code,validity_date,ghg_total_without_lulucf,ghg_total_with_lulucf,gdp,gdp_ppp
144,ARG,2012,338963349.93,429478449.93,545982375701.128,819697901873.041
145,ARG,2010,332498160.0,447642430.0,423627422092.49,736718360687.747
224,ARG,2019,0.0,0.0,451932356085.842,1033557797795.68
225,ARG,1972,0.0,0.0,34733000536.2862,0.0
226,ARG,1999,0.0,0.0,283523000000.0,422359326228.804
227,ARG,1989,0.0,0.0,76636898036.4712,0.0
228,ARG,1979,0.0,0.0,69252328953.3789,0.0
229,ARG,1965,0.0,0.0,28344705966.6389,0.0
621,ARG,1990,231057275.548153,216291389.487407,141352368714.691,234564125672.49
622,ARG,1994,257522426.252628,223335527.904266,257440000000.0,349832146514.708


In [18]:
#df_unfccc = dequantify_df(df_unfccc)
#df_unfccc
#df_unfccc.info(verbose=True)
#df_unfccc

df_result= requantify_df(df_result)
#df_result= dequantify_df(df_result)
#df_result


<font size=3><b>Merge UNFCCC , Worldbank and OECD data</b> </font>

In [19]:
#df_result.info(verbose=True)
#df_result= requantify_df(df_result)
#df_result.info(verbose=True)

#df_unfccc = df_unfccc.assign(ghg_total_without_lulucf=np.where(df_unfccc.ghg_total_without_lulucf.isnull(), 0, df_unfccc.ghg_total_without_lulucf))
#df_unfccc = df_unfccc.assign(ghg_total_with_lulucf=np.where(df_unfccc.ghg_total_with_lulucf.isnull(), 0, df_unfccc.ghg_total_with_lulucf))
#df_unfccc = df_unfccc.assign(gdp=np.where(df_unfccc.gdp.isnull(), 0, df_unfccc.gdp))
#df_unfccc = df_unfccc.assign(gdp_ppp=np.where(df_unfccc.gdp_ppp.isnull(), 0, df_unfccc.gdp_ppp))


df_result = pd.merge(df_result,df_unfccc,on=['country_iso_code','validity_date'],how='outer')  


rename_columns = {'ghg_total_without_lulucf':'scope1_excl_lulucf',
                  'ghg_total_with_lulucf':'scope1_incl_lulucf'}
#columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_lulucf_units','scope1_incl_lulucf','scope1_incl_lulucf_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','exported_emissions','exported_emissions_units','gdp_ppp','gdp_ppp_units','gdp','gdp_units']
df_result = df_result.rename(columns=rename_columns)
df_result.info(verbose=True)

# enrich with population data

df_result = pd.merge(df_result,df_population,on=['country_iso_code','validity_date'],how='outer')  


rename_columns = {'value':'population'}
#columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_excl_lulucf_units','scope1_incl_lulucf','scope1_incl_lulucf_units','scope2_value','scope2_value_units','scope3_value','scope3_value_units','exported_emissions','exported_emissions_units','gdp_ppp','gdp_ppp_units','gdp','gdp_units']
df_result = df_result.rename(columns=rename_columns)
df_result.info(verbose=True)
df_result["population"] = df_result["population"].fillna(0)



#df_result['scope1_excl_lulucf'] = df_result['scope1_excl_lulucf'].fillna(0).astype(float)
#df_result['scope1_incl_lulucf'] = df_result['scope1_incl_lulucf'].fillna(0).astype(float)
#df_result['scope1_excl_lulucf'] = df_result['scope1_excl_lulucf'].fillna(0).astype(float)
#df_result['scope2_value'] = df_result['scope2_value'].fillna(0).astype(float)
#df_result['scope3_value'] = df_result['scope3_value'].fillna(0).astype(float)



df_result['sum_yn'] = 1 * ((df_result['scope1_excl_lulucf'] > 0) & (df_result['scope2_value'] > 0) &  (df_result['scope3_value'] > 0) )
df_result['calc_attr_factor_yn'] = 1 * (df_result['gdp_ppp'] > 0)
df_result['population_available'] = 1 * (df_result['population'] > 0)


df_result['consumption_emissions_excl_lulucf'] = df_result['sum_yn'] * ( df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value'] - df_result['exported_emissions'])
df_result['consumption_emissions_incl_lulucf'] = df_result['sum_yn'] * ( df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value'] - df_result['exported_emissions'])

df_result['consumption_emissions_excl_lulucf_per_capita']  = df_result['population_available'] * df_result['consumption_emissions_excl_lulucf'] / df_result['population']
df_result['consumption_emissions_incl_lulucf_per_capita']  = df_result['population_available'] * df_result['consumption_emissions_incl_lulucf'] / df_result['population']


###############
df_result['attribution_factor_scope1_excl_lulucf']= df_result['calc_attr_factor_yn'] * (df_result['scope1_excl_lulucf']) / df_result['gdp_ppp']
df_result['attribution_factor_scope1_incl_lulucf']= df_result['calc_attr_factor_yn'] * (df_result['scope1_incl_lulucf']) / df_result['gdp_ppp']

df_result['attribution_factor_excl_lulucf']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp_ppp'])
df_result['attribution_factor_incl_lulucf']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp_ppp'])
##########
df_result['attribution_factor_excl_lulucf_gdp']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_excl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp'])
df_result['attribution_factor_incl_lulucf_gdp']= df_result['calc_attr_factor_yn'] * df_result['sum_yn'] * ( (df_result['scope1_incl_lulucf'] + df_result['scope2_value'] + df_result['scope3_value']) / df_result['gdp'])



#df_result['ghg_total_without_lulucf']=df_result['ghg_total_without_lulucf'].replace(0, np.nan)
#df_result['ghg_total_with_lulucf']=df_result['ghg_total_with_lulucf'].replace(0, np.nan)
#df_result['gdp']=df_result['gdp'].replace(0, np.nan)
#df_result['gdp_ppp']=df_result['gdp_ppp'].replace(0, np.nan)



#df_result

columns_order = ['country_iso_code','validity_date','scope1_excl_lulucf','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'population','consumption_emissions_excl_lulucf_per_capita','consumption_emissions_incl_lulucf_per_capita',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result = df_result.reindex(columns=columns_order)
df_result.info(verbose=True)
df_result= dequantify_df(df_result)
df_result.info(verbose=True)
# remove invalid country_iso_codes (regions, ...)

df_result = pd.merge(df_result,df_country,on=['country_iso_code'])  

cols= ['scope1_excl_lulucf','scope1_incl_lulucf','scope2_value','scope3_value',
                 'exported_emissions','gdp_ppp','gdp',
                 'consumption_emissions_excl_lulucf','consumption_emissions_incl_lulucf',
                 'population','consumption_emissions_excl_lulucf_per_capita','consumption_emissions_incl_lulucf_per_capita',
                 'attribution_factor_scope1_excl_lulucf','attribution_factor_scope1_incl_lulucf',
                 'attribution_factor_excl_lulucf','attribution_factor_excl_lulucf_gdp',
                 'attribution_factor_incl_lulucf','attribution_factor_incl_lulucf_gdp']
df_result[cols] = df_result[cols].replace({0:np.nan})


df_result


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13380 entries, 0 to 13379
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   country_iso_code    13380 non-null  string         
 1   validity_date       13380 non-null  Int64          
 2   scope2_value        2016 non-null   pint[CO2eq * t]
 3   scope3_value        2016 non-null   pint[CO2eq * t]
 4   exported_emissions  2016 non-null   pint[CO2eq * t]
 5   scope1_excl_lulucf  12948 non-null  pint[CO2eq * t]
 6   scope1_incl_lulucf  12948 non-null  pint[CO2eq * t]
 7   gdp                 12948 non-null  pint[USD]      
 8   gdp_ppp             12948 non-null  pint[USD]      
dtypes: Int64(1), pint[CO2eq * t](5), pint[USD](2), string(1)
memory usage: 1.0 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13791 entries, 0 to 13790
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype          
---  ------     

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,...,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
0,ARG,1995,NaN,CO2eq * t,NaN,CO2eq * t,21000.0,CO2eq * t,9356000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
1,ARG,1996,NaN,CO2eq * t,NaN,CO2eq * t,28000.0,CO2eq * t,11287000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
2,ARG,1997,2.709103e+08,CO2eq * t,2.419562e+08,CO2eq * t,26000.0,CO2eq * t,14968000.0,CO2eq * t,...,0.00059,CO2eq * t / USD,0.000697,CO2eq * t / USD,0.000976,CO2eq * t / USD,0.000626,CO2eq * t / USD,0.000877,CO2eq * t / USD
3,ARG,1998,NaN,CO2eq * t,NaN,CO2eq * t,43000.0,CO2eq * t,13397000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
4,ARG,1999,NaN,CO2eq * t,NaN,CO2eq * t,61000.0,CO2eq * t,12270000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10585,MAF,2020,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
10586,MAF,2017,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
10587,MAF,2016,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
10588,MAF,2018,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD


In [20]:
#df_result['scope1_value_units'] = 'Mt CO2e'
#df_result['scope1_value'] = df_result['scope1_value'].round(decimals = 3)
#pd.options.display.float_format = '{:.3f}'.format
#df_result= df_result.convert_dtypes()
#df_result.info(verbose=True)
df_result[df_result['country_iso_code']=='CAN']
#df_result

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,...,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
407,CAN,1995,6.562987e+08,CO2eq * t,6.265742e+08,CO2eq * t,5129000.0,CO2eq * t,179111000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
408,CAN,1996,6.787145e+08,CO2eq * t,6.419787e+08,CO2eq * t,5269000.0,CO2eq * t,188198000.0,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
409,CAN,1997,6.911954e+08,CO2eq * t,6.529372e+08,CO2eq * t,6362000.0,CO2eq * t,202972000.0,CO2eq * t,...,0.000864,CO2eq * t / USD,0.001192,CO2eq * t / USD,0.001375,CO2eq * t / USD,0.001141,CO2eq * t / USD,0.001316,CO2eq * t / USD
410,CAN,1998,6.969015e+08,CO2eq * t,6.470244e+08,CO2eq * t,7993000.0,CO2eq * t,214108000.0,CO2eq * t,...,0.000815,CO2eq * t / USD,0.001158,CO2eq * t / USD,0.001450,CO2eq * t / USD,0.001095,CO2eq * t / USD,0.001371,CO2eq * t / USD
411,CAN,1999,7.096009e+08,CO2eq * t,6.742546e+08,CO2eq * t,9100000.0,CO2eq * t,222868000.0,CO2eq * t,...,0.000797,CO2eq * t / USD,0.001112,CO2eq * t / USD,0.001388,CO2eq * t / USD,0.001071,CO2eq * t / USD,0.001336,CO2eq * t / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,CAN,1970,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
465,CAN,1967,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
466,CAN,1981,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD
467,CAN,1988,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,NaN,CO2eq * t,...,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD,NaN,CO2eq * t / USD


Calculate Attribution factor = (Scope1+Scope2+Scope3)/GDP


In [21]:


#df_result= requantify_df(df_result)
format_mapper =  {'gdp_ppp': '{0:,.0f}',
           'gdp': '{0:,.0f}',       
           'attribution_factor_excl_lulucf': '{0:.8f}',
           'attribution_factor_incl_lulucf': '{0:.8f}',
           'attribution_factor_excl_lulucf_gdp': '{0:.8f}',
           'attribution_factor_incl_lulucf_gdp': '{0:.8f}',       
           'scope1_excl_lulucf': '{0:,.0f}',       
           'scope1_incl_lulucf': '{0:,.0f}',              
           'scope2_value': '{0:,.0f}',              
           'scope3_value': '{0:,.0f}'
                 }

df_result[df_result['country_iso_code']=='CAN'].style.format(format_mapper)


,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,exported_emissions,exported_emissions_units,gdp_ppp,gdp_ppp_units,gdp,gdp_units,consumption_emissions_excl_lulucf,consumption_emissions_excl_lulucf_units,consumption_emissions_incl_lulucf,consumption_emissions_incl_lulucf_units,population,consumption_emissions_excl_lulucf_per_capita,consumption_emissions_excl_lulucf_per_capita_units,consumption_emissions_incl_lulucf_per_capita,consumption_emissions_incl_lulucf_per_capita_units,attribution_factor_scope1_excl_lulucf,attribution_factor_scope1_excl_lulucf_units,attribution_factor_scope1_incl_lulucf,attribution_factor_scope1_incl_lulucf_units,attribution_factor_excl_lulucf,attribution_factor_excl_lulucf_units,attribution_factor_excl_lulucf_gdp,attribution_factor_excl_lulucf_gdp_units,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units
407,CAN,1995,"656,298,717",CO2eq * t,"626,574,181",CO2eq * t,"5,129,000",CO2eq * t,"179,111,000",CO2eq * t,136648000.000000,CO2eq * t,nan,USD,"604,031,623,433",USD,703890717.315296,CO2eq * t,674166180.603995,CO2eq * t,,nan,CO2eq * t,nan,CO2eq * t,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD
408,CAN,1996,"678,714,489",CO2eq * t,"641,978,685",CO2eq * t,"5,269,000",CO2eq * t,"188,198,000",CO2eq * t,139744000.000000,CO2eq * t,nan,USD,"628,546,387,972",USD,732437488.508001,CO2eq * t,695701685.136717,CO2eq * t,,nan,CO2eq * t,nan,CO2eq * t,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD,nan,CO2eq * t / USD
409,CAN,1997,"691,195,367",CO2eq * t,"652,937,162",CO2eq * t,"6,362,000",CO2eq * t,"202,972,000",CO2eq * t,160734000.000000,CO2eq * t,"755,390,170,461",USD,"654,986,999,856",USD,739795367.182711,CO2eq * t,701537162.403060,CO2eq * t,,nan,CO2eq * t,nan,CO2eq * t,0.000915,CO2eq * t / USD,0.000864,CO2eq * t / USD,0.00119214,CO2eq * t / USD,0.00137488,CO2eq * t / USD,0.00114149,CO2eq * t / USD,0.00131647,CO2eq * t / USD
410,CAN,1998,"696,901,519",CO2eq * t,"647,024,422",CO2eq * t,"7,993,000",CO2eq * t,"214,108,000",CO2eq * t,166623000.000000,CO2eq * t,"793,674,327,704",USD,"634,000,000,000",USD,752379518.588772,CO2eq * t,702502422.222185,CO2eq * t,,nan,CO2eq * t,nan,CO2eq * t,0.000878,CO2eq * t / USD,0.000815,CO2eq * t / USD,0.00115791,CO2eq * t / USD,0.00144953,CO2eq * t / USD,0.00109507,CO2eq * t / USD,0.00137086,CO2eq * t / USD
411,CAN,1999,"709,600,924",CO2eq * t,"674,254,607",CO2eq * t,"9,100,000",CO2eq * t,"222,868,000",CO2eq * t,171505000.000000,CO2eq * t,"846,412,945,810",USD,"678,412,196,271",USD,770063924.294885,CO2eq * t,734717607.218812,CO2eq * t,,nan,CO2eq * t,nan,CO2eq * t,0.000838,CO2eq * t / USD,0.000797,CO2eq * t / USD,0.00111242,CO2eq * t / USD,0.00138790,CO2eq * t / USD,0.00107066,CO2eq * t / USD,0.00133580,CO2eq * t / USD
412,CAN,2000,"733,511,337",CO2eq * t,"711,770,284",CO2eq * t,"18,191,000",CO2eq * t,"236,036,000",CO2eq * t,178858000.000000,CO2eq * t,"900,996,986,801",USD,"744,773,415,932",USD,808880337.261223,CO2eq * t,787139284.018432,CO2eq * t,30685730,26.360146,CO2eq * t,25.651640,CO2eq * t,0.000814,CO2eq * t / USD,0.000790,CO2eq * t / USD,0.00109627,CO2eq * t / USD,0.00132623,CO2eq * t / USD,0.00107214,CO2eq * t / USD,0.00129704,CO2eq * t / USD
413,CAN,2001,"722,935,012",CO2eq * t,"685,252,427",CO2eq * t,"19,531,000",CO2eq * t,"228,887,000",CO2eq * t,175811000.000000,CO2eq * t,"937,786,776,177",USD,"738,981,792,355",USD,795542012.117626,CO2eq * t,757859426.599736,CO2eq * t,,nan,CO2eq * t,nan,CO2eq * t,0.000771,CO2eq * t / USD,0.000731,CO2eq * t / USD,0.00103579,CO2eq * t / USD,0.00131445,CO2eq * t / USD,0.00099561,CO2eq * t / USD,0.00126346,CO2eq * t / USD
414,CAN,2002,"726,555,183",CO2eq * t,"706,282,510",CO2eq * t,"8,195,000",CO2eq * t

<font size=3>Save the results in Trino</font>

In [22]:
# Add the sources used for the calculations in the result table
df_result['scope1_excl_source'] = "UNFCCC -> Time_Series_GHG_total_without_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope1_incl_source'] = "UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx"
df_result['scope2_source'] = "OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE"
df_result['scope3_source'] = "OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value "          
df_result['gdp_ppp_source'] = "Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv"          


In [23]:
import osc_ingest_trino as osc
#df_result['validity_date']=pd.to_datetime(df_result['validity_date'], unit='D')
df_result=df_result.convert_dtypes()
df_result.info(verbose=True)

df_result['year'] = df_result['validity_date']
#df_result['validity_date'] = pd.to_datetime(df_result.validity_date, format='%Y')
df_result = df_result.convert_dtypes()
#df_result.validity_date = df_result.validity_date.astype('datetime64[ns]')



columnschema = osc.create_table_schema_pairs(df_result,typemap={'datetime64[ns]':'timestamp(6)'}) 
sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())





<class 'pandas.core.frame.DataFrame'>
Int64Index: 10590 entries, 0 to 10589
Data columns (total 42 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   country_iso_code                                    10590 non-null  string 
 1   validity_date                                       10590 non-null  Int64  
 2   scope1_excl_lulucf                                  2212 non-null   Float64
 3   scope1_excl_lulucf_units                            10590 non-null  string 
 4   scope1_incl_lulucf                                  2212 non-null   Float64
 5   scope1_incl_lulucf_units                            10590 non-null  string 
 6   scope2_value                                        1419 non-null   Float64
 7   scope2_value_units                                  10590 non-null  string 
 8   scope3_value                                        1608 non-null   Int64  


In [24]:
df_result.info("verbose=True")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10590 entries, 0 to 10589
Data columns (total 43 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   country_iso_code                                    10590 non-null  string 
 1   validity_date                                       10590 non-null  Int64  
 2   scope1_excl_lulucf                                  2212 non-null   Float64
 3   scope1_excl_lulucf_units                            10590 non-null  string 
 4   scope1_incl_lulucf                                  2212 non-null   Float64
 5   scope1_incl_lulucf_units                            10590 non-null  string 
 6   scope2_value                                        1419 non-null   Float64
 7   scope2_value_units                                  10590 non-null  string 
 8   scope3_value                                        1608 non-null   Int64  


In [25]:
df_result = df_result[df_result['year']>=1990]
df_result.sort_values(by=['country_iso_code','validity_date'], inplace=True)
df_result.to_excel("pcaf_results.xls",index=False,encoding='utf-8')


/opt/app-root/lib64/python3.8/site-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
/tmp/ipykernel_1176/995226512.py:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_result.to_excel("pcaf_results.xls",index=False,encoding='utf-8')


In [26]:
df_result.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 50 , verbose = True))


constructed fully qualified table name as: "pcaf_sovereign_footprint.sf_total_sovereign_emissions"
inserting 50 records
  ('ABW', 1990, NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t', 1447708861.20673, 'USD', 764887117.194486, 'USD', NULL, 'CO2eq * t', NULL, 'CO2eq * t', 62152, NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', NULL, 'CO2eq * t / USD', 'UNFCCC -> Time_Series_GHG_total_without_LULUCF_in_kt_CO2_equivalent.xlsx', 'UNFCCC -> Time_Series_GHG_total_with_LULUCF_in_kt_CO2_equivalent.xlsx', 'OECD -> IMGR_FCO2.csv -> Industry_Code = D35  -> Aggregation grouped by Partner_ISO_CODE', 'OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL - Scope_2 Value ', 'Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4019306.csv', 1990)
  ('ABW', 1991, NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t', NULL, 'CO2eq * t', 

In [27]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table} where country_iso_code = 'CAN' and year = 2018 """
pd.read_sql(sql, engine)

,country_iso_code,validity_date,scope1_excl_lulucf,scope1_excl_lulucf_units,scope1_incl_lulucf,scope1_incl_lulucf_units,scope2_value,scope2_value_units,scope3_value,scope3_value_units,...,attribution_factor_incl_lulucf,attribution_factor_incl_lulucf_units,attribution_factor_incl_lulucf_gdp,attribution_factor_incl_lulucf_gdp_units,scope1_excl_source,scope1_incl_source,scope2_source,scope3_source,gdp_ppp_source,year
0,CAN,2018,7.284759e+08,CO2eq * t,7.368870e+08,CO2eq * t,4953000.0,CO2eq * t,260931000,CO2eq * t,...,0.000541,CO2eq * t / USD,0.000581,CO2eq * t / USD,UNFCCC -> Time_Series_GHG_total_without_LULUCF...,UNFCCC -> Time_Series_GHG_total_with_LULUCF_in...,OECD -> IMGR_FCO2.csv -> Industry_Code = D35 ...,OECD -> IMGR_FCO2.csv -> Indutry_Code = DTOTAL...,Worldbank -> API_NY.GDP.MKTP.CD_DS2_en_csv_v2_...,2018
